In [ ]:
!kill -KILL 300065

In [ ]:
!nvidia-smi

In [2]:
from inference_run import run_inference

In [ ]:
run_inference(
    experiment="gemma-2b-it",
    run_name="gsm8k-gt/m123-m4f-tf/m123",
    postfix="_test",
    #previous_next_steps="/cluster/work/lawecon/Work/mbraasch/output/gemma-2b-it/gsm8k-dl/m123-m4f-tf/m123/s1_next_step_predictions_s1_test.json",
    #data_dir = "/cluster/work/lawecon/Work/mbraasch/data/gsm8k_train.json"
)